In [4]:
import pandas as pd
import json

# JSON fornecido
json_data = '''{
  "guestChecks": [
    {
      "guestCheckId": 1122334455,
      "chkNum": 1234,
      "opnBusDt": "2024-01-01",
      "opnUTC": "2024-01-01T09:09:09",
      "opnLcl": "2024-01-01T06:09:09",
      "clsdBusDt": "2024-01-01",
      "clsdUTC": "2024-01-01T12:12:12",
      "clsdLcl": "2024-01-01T09:12:12",
      "lastTransUTC": "2024-01-01T12:12:12",
      "lastTransLcl": "2024-01-01T09:12:12",
      "lastUpdatedUTC": "2024-01-01T13:13:13",
      "lastUpdatedLcl": "2024-01-01T10:13:13",
      "clsdFlag": true,
      "gstCnt": 1,
      "subTtl": 109.9,
      "nonTxblSlsTtl": null,
      "chkTtl": 109.9,
      "dscTtl": -10,
      "payTtl": 109.9,
      "balDueTtl": null,
      "rvcNum": 101,
      "otNum": 1,
      "ocNum": null,
      "tblNum": 1,
      "tblName": "90",
      "empNum": 55555,
      "numSrvcRd": 3,
      "numChkPrntd": 2,
      "taxes": [
        {
          "taxNum": 28,
          "txblSlsTtl": 119.9,
          "taxCollTtl": 20.81,
          "taxRate": 21,
          "type": 3
        }
      ],
      "detailLines": [
        {
          "guestCheckLineItemId": 9988776655,
          "rvcNum": 123,
          "dtlOtNum": 1,
          "dtlOcNum": null,
          "lineNum": 1,
          "dtlId": 1,
          "detailUTC": "2024-01-01T09:09:09",
          "detailLcl": "2024-01-01T06:09:09",
          "lastUpdateUTC": "2024-11-01T10:10:10",
          "lastUpdateLcl": "2024-01-01T07:10:10",
          "busDt": "2024-01-01",
          "wsNum": 7,
          "dspTtl": 119.9,
          "dspQty": 1,
          "aggTtl": 119.9,
          "aggQty": 1,
          "chkEmpId": 10454318,
          "chkEmpNum": 81001,
          "svcRndNum": 1,
          "seatNum": 1,
          "menuItem": {
            "miNum": 6042,
            "modFlag": false,
            "inclTax": 20.809091,
            "activeTaxes": "28",
            "prcLvl": 3
          }
        }
      ]
    }
  ]
}'''

# Carregar JSON
data = json.loads(json_data)

# Função para renomear colunas dos DataFrames
def rename_columns(df, columns_dict):
    df.rename(columns=columns_dict, inplace=True)

# Criar DataFrame para GuestCheck
guest_checks = pd.json_normalize(data['guestChecks'])
guest_checks_columns = {
    'guestCheckId': 'Guest Check ID',
    'chkNum': 'Check Number',
    'opnBusDt': 'Open Business Date',
    'opnUTC': 'Open UTC',
    'opnLcl': 'Open Local',
    'clsdBusDt': 'Closed Business Date',
    'clsdUTC': 'Closed UTC',
    'clsdLcl': 'Closed Local',
    'lastTransUTC': 'Last Transaction UTC',
    'lastTransLcl': 'Last Transaction Local',
    'lastUpdatedUTC': 'Last Updated UTC',
    'lastUpdatedLcl': 'Last Updated Local',
    'clsdFlag': 'Closed Flag',
    'gstCnt': 'Guest Count',
    'subTtl': 'Subtotal',
    'nonTxblSlsTtl': 'Non-Taxable Sales Total',
    'chkTtl': 'Check Total',
    'dscTtl': 'Discount Total',
    'payTtl': 'Payment Total',
    'balDueTtl': 'Balance Due Total',
    'rvcNum': 'Revenue Center Number',
    'otNum': 'Order Type Number',
    'ocNum': 'Order Control Number',
    'tblNum': 'Table Number',
    'tblName': 'Table Name',
    'empNum': 'Employee Number',
    'numSrvcRd': 'Number of Service Rounds',
    'numChkPrntd': 'Number of Checks Printed'
}
rename_columns(guest_checks, guest_checks_columns)
print("GuestCheck DataFrame:")
display(guest_checks)

# Criar DataFrame para Taxes
taxes = pd.json_normalize(data['guestChecks'], 'taxes', ['guestCheckId'])
taxes_columns = {
    'taxNum': 'Tax Number',
    'guestCheckId': 'Guest Check ID',
    'txblSlsTtl': 'Taxable Sales Total',
    'taxCollTtl': 'Tax Collected Total',
    'taxRate': 'Tax Rate',
    'type': 'Type'
}
rename_columns(taxes, taxes_columns)
print("\nTaxes DataFrame:")
display(taxes)

# Criar DataFrame para DetailLines
detail_lines = pd.json_normalize(data['guestChecks'], 'detailLines', ['guestCheckId'])
detail_lines_columns = {
    'guestCheckLineItemId': 'Guest Check Line Item ID',
    'guestCheckId': 'Guest Check ID',
    'rvcNum': 'Revenue Center Number',
    'dtlOtNum': 'Detail Order Type Number',
    'dtlOcNum': 'Detail Order Control Number',
    'lineNum': 'Line Number',
    'dtlId': 'Detail ID',
    'detailUTC': 'Detail UTC',
    'detailLcl': 'Detail Local',
    'lastUpdateUTC': 'Last Update UTC',
    'lastUpdateLcl': 'Last Update Local',
    'busDt': 'Business Date',
    'wsNum': 'Workstation Number',
    'dspTtl': 'Display Total',
    'dspQty': 'Display Quantity',
    'aggTtl': 'Aggregate Total',
    'aggQty': 'Aggregate Quantity',
    'chkEmpId': 'Check Employee ID',
    'chkEmpNum': 'Check Employee Number',
    'svcRndNum': 'Service Round Number',
    'seatNum': 'Seat Number'
}
rename_columns(detail_lines, detail_lines_columns)
print("\nDetailLines DataFrame:")
display(detail_lines)

# Extrair MenuItem de DetailLines
menu_items_list = []
for detail in data['guestChecks'][0]['detailLines']:
    menu_item = detail.pop('menuItem')
    menu_item['guestCheckId'] = data['guestChecks'][0]['guestCheckId']
    menu_item['guestCheckLineItemId'] = detail['guestCheckLineItemId']
    menu_items_list.append(menu_item)

menu_items = pd.DataFrame(menu_items_list)
menu_items_columns = {
    'miNum': 'Menu Item Number',
    'modFlag': 'Modification Flag',
    'inclTax': 'Included Tax',
    'activeTaxes': 'Active Taxes',
    'prcLvl': 'Price Level',
    'guestCheckId': 'Guest Check ID',
    'guestCheckLineItemId': 'Guest Check Line Item ID'
}
rename_columns(menu_items, menu_items_columns)
print("\nMenuItem DataFrame:")
display(menu_items)


GuestCheck DataFrame:


,Guest Check ID,Check Number,Open Business Date,Open UTC,Open Local,Closed Business Date,Closed UTC,Closed Local,Last Transaction UTC,Last Transaction Local,...,Revenue Center Number,Order Type Number,Order Control Number,Table Number,Table Name,Employee Number,Number of Service Rounds,Number of Checks Printed,taxes,detailLines
0,1122334455,1234,2024-01-01,2024-01-01T09:09:09,2024-01-01T06:09:09,2024-01-01,2024-01-01T12:12:12,2024-01-01T09:12:12,2024-01-01T12:12:12,2024-01-01T09:12:12,...,101,1,None,1,90,55555,3,2,"[{'taxNum': 28, 'txblSlsTtl': 119.9, 'taxCollT...","[{'guestCheckLineItemId': 9988776655, 'rvcNum'..."



Taxes DataFrame:


,Tax Number,Taxable Sales Total,Tax Collected Total,Tax Rate,Type,Guest Check ID
0,28,119.9,20.81,21,3,1122334455



DetailLines DataFrame:


,Guest Check Line Item ID,Revenue Center Number,Detail Order Type Number,Detail Order Control Number,Line Number,Detail ID,Detail UTC,Detail Local,Last Update UTC,Last Update Local,...,Check Employee ID,Check Employee Number,Service Round Number,Seat Number,menuItem.miNum,menuItem.modFlag,menuItem.inclTax,menuItem.activeTaxes,menuItem.prcLvl,Guest Check ID
0,9988776655,123,1,None,1,1,2024-01-01T09:09:09,2024-01-01T06:09:09,2024-11-01T10:10:10,2024-01-01T07:10:10,...,10454318,81001,1,1,6042,False,20.809091,28,3,1122334455



MenuItem DataFrame:


,Menu Item Number,Modification Flag,Included Tax,Active Taxes,Price Level,Guest Check ID,Guest Check Line Item ID
0,6042,False,20.809091,28,3,1122334455,9988776655
